In [ ]:
"Créditos: La arquitectura de red neuronal unet_attention fue modificada de Nikhil Tomar en su repositorio GitHub: https://github.com/nikhilroxtomar/Semantic-Segmentation-Architecture/blob/main/TensorFlow/attention-unet.py " 

In [ ]:
import tensorflow as tf
import tensorflow.keras.layers as L
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.metrics import BinaryAccuracy, BinaryIoU
import os
import matplotlib.pyplot as plt
import cv2
import numpy as np
from skimage.transform import resize
from sklearn.model_selection import train_test_split
import pandas as pd

Cargue de datos.

In [ ]:
# Rutas de las carpetas
carpeta_fallas = "/kaggle/input/implem-fase2/fase2/Fallas"
carpeta_sismica = "/kaggle/input/implem-fase2/fase2/Sismica"


archivos_fallas = sorted(os.listdir(carpeta_fallas))
archivos_sismica = sorted(os.listdir(carpeta_sismica))

fallas_data = []
sismica_data = []

for archivo_falla, archivo_sismica in zip(archivos_fallas, archivos_sismica):
    if archivo_falla == archivo_sismica:  # Verifica que los nombres coincidan
        # Cargar los arrays
        falla_array = np.load(os.path.join(carpeta_fallas, archivo_falla))
        sismica_array = np.load(os.path.join(carpeta_sismica, archivo_sismica))

        # Asegurarse de que las imágenes tengan la misma forma
        if falla_array.shape != (128, 128):
            falla_array = resize(falla_array, (128, 128))
        if sismica_array.shape != (128, 128):
            sismica_array = resize(sismica_array, (128, 128))

        # Añadir a las listas
        fallas_data.append(falla_array)
        sismica_data.append(sismica_array)
    else:
        print(f"Advertencia: Los archivos no coinciden: {archivo_falla} vs {archivo_sismica}")

images = np.array(sismica_data)
masks = np.array(fallas_data)




In [ ]:
#Dividir dataset
images_train, images_test, masks_train, masks_test = train_test_split(images, masks, random_state=13, test_size=0.2)
images_train = np.nan_to_num(images_train, nan=0.0)
images_test = np.nan_to_num(images_test, nan=0.0)

In [ ]:
np.save("mascarastest.npy", masks_test)

In [ ]:
#Visualizar los datos
def visualize_data(images, masks, num_samples=5):
    plt.figure(figsize=(12, 5))

    for i in range(num_samples):
        # Mostrar la imagen
        plt.subplot(2, num_samples, i + 1)
        plt.imshow(images[i+1].T, cmap='gray')
        plt.title(f'Imagen {i + 1}')
        plt.axis('off')
        plt.colorbar()


        plt.subplot(2, num_samples, i + 1 + num_samples)
        plt.imshow(masks[i+1].T, cmap='gist_gray', interpolation='Nearest')
        plt.title(f'Máscara {i + 1}')
        plt.axis('off')
        plt.colorbar()

    plt.savefig("patches_unet_att.svg", format="svg")
    plt.show()

visualize_data(images_train, masks_train, num_samples=4)

Modelo Unet complementado con Bloques de Atención.

In [ ]:

def conv_block(x, num_filters):
    x = L.Conv2D(num_filters, 3, padding="same")(x)
    x = L.BatchNormalization()(x)
    x = L.Activation("relu")(x)

    x = L.Conv2D(num_filters, 3, padding="same")(x)
    x = L.BatchNormalization()(x)
    x = L.Activation("relu")(x)

    return x

def encoder_block(x, num_filters):
    x = conv_block(x, num_filters)
    p = L.MaxPool2D((2, 2))(x)
    return x, p

def attention_gate(g, s, num_filters):
    Wg = L.Conv2D(num_filters, 1, padding="same")(g)
    Wg = L.BatchNormalization()(Wg)

    Ws = L.Conv2D(num_filters, 1, padding="same")(s)
    Ws = L.BatchNormalization()(Ws)

    out = L.Activation("relu")(Wg + Ws)
    out = L.Conv2D(num_filters, 1, padding="same")(out)
    out = L.Activation("sigmoid")(out)

    return out * s

def decoder_block(x, s, num_filters):
    x = L.UpSampling2D(interpolation="bilinear")(x)
    s = attention_gate(x, s, num_filters)
    x = L.Concatenate()([x, s])
    x = conv_block(x, num_filters)
    return x

def attention_unet(input_shape):
    """ Inputs """
    inputs = L.Input(input_shape)

    """ Encoder """
    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)

    b1 = conv_block(p3, 512)

    """ Decoder """
    d1 = decoder_block(b1, s3, 256)
    d2 = decoder_block(d1, s2, 128)
    d3 = decoder_block(d2, s1, 64)

    """ Outputs """
    outputs = L.Conv2D(1, 1, padding="same", activation="sigmoid")(d3)

    """ Model """
    model = Model(inputs, outputs, name="Attention-UNET")
    return model

if __name__ == "__main__":
    input_shape = (128, 128, 1)
    model = attention_unet(input_shape)
    model.summary()

Hiperparámetros

In [ ]:
#Hiperparámetros configurables
lr = 1e-4
batch_size = 32
epochs = 25
#Compilar modelo
optimizer= tf.optimizers.Adam(learning_rate=lr)
loss_fn= tf.keras.losses.BinaryCrossentropy(from_logits=False)
metric = [tf.keras.metrics.Precision(),BinaryIoU(target_class_ids=[1], threshold=0.5, name=None, dtype=None), BinaryAccuracy(dtype=None, threshold=0.5)]
model.compile(optimizer=optimizer, loss=loss_fn, metrics=metric)

Entrenamiento de la red.

In [ ]:
images_train.shape

In [ ]:
images_test.shape

In [ ]:
print(np.min(images_train))
print(np.max(images_train))

In [ ]:
historia = model.fit(images_train, masks_train, epochs=epochs, batch_size=batch_size, shuffle=True, validation_split=0.2)

DESEMPEÑO DE LA RED DURANTE EL ENTRENAMIENTO

In [ ]:
#Metricas de entrenamiento
Loss = historia.history['loss']
IoU = historia.history['binary_io_u']
Precision = historia.history['precision']
Accuracy = historia.history['binary_accuracy']


#Métricas de validación
Val_Loss = historia.history['val_loss']
Val_IoU = historia.history['val_binary_io_u']
Val_Precision = historia.history['val_precision']
Val_Accuracy = historia.history['val_binary_accuracy']


In [ ]:
import csv
with open("training_metrics_unet_att.csv", "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["Epoch", "Loss", "Accuracy"])  # Cabecera
    for Loss, IoU, Precision, Accuracyy in zip( Loss, IoU, Precision, Accuracy):
        writer.writerow([Loss, IoU, Precision, Accuracy])

In [ ]:
#Grafica de Pérdida
epochs_range = range(1, len(Loss) + 1)
plt.figure(figsize=(7, 3))
plt.plot(epochs_range, Loss, color='red', label='Entrenamiento')
plt.plot(epochs_range, Val_Loss, color='blue', label= 'Validacion')
plt.ylim(0,1)
plt.xlabel('Épocas')
plt.ylabel('Pérdida')
plt.legend()
plt.savefig("loss_unet_att.svg", format="svg")
plt.show()


In [ ]:
#Gráfica de Precisión
plt.figure(figsize=(7, 3))
plt.plot(epochs_range, Precision, color='red', label='Entrenamiento')
plt.plot(epochs_range, Val_Precision, color='blue', label='Validación')
plt.ylim(0,1)
plt.xlabel('Épocas')
plt.ylabel('Precision')
plt.legend()
plt.savefig("precision_unet_att.svg", format="svg")
plt.show()


In [ ]:
#Gráfica de IoU
plt.figure(figsize=(7, 3))
plt.plot(epochs_range, IoU, color='red', label='Entrenamiento')
plt.plot(epochs_range, Val_IoU, color='blue', label='Validación')
plt.ylim(0,1)
plt.xlabel('Épocas')
plt.ylabel('IoU')
plt.legend()
plt.savefig("IoU_unet_att.svg", format="svg")
plt.show()


In [ ]:
#Gráfica de Accuracy
plt.figure(figsize=(7, 3))
plt.plot(epochs_range, Accuracy, color='red', label='Entrenamiento')
plt.plot(epochs_range, Val_Accuracy, color='blue', label='Validación')
plt.ylim(0,1)
plt.xlabel('Épocas')
plt.ylabel('Exactitud')
plt.legend()
plt.savefig("Accuracy_unet_att.svg", format="svg")
plt.show()


TESTEAR EL MODELO

In [ ]:
predicciones = model.predict(images_test)

In [ ]:
#Evaluar modelo con los datos de test.
historia2 = model.evaluate(
    x=images_test,
    y=masks_test,
    batch_size=None,
    verbose="auto",
    sample_weight=None,
    steps=None,
    callbacks=None,
    return_dict=False,)

In [ ]:
def mostrar_resultados(imagenes, mascaras_reales, mascaras_predichas, num_ejemplos=5):
    plt.figure(figsize=(15, num_ejemplos * 5))
    
    for i in range(num_ejemplos):
        
        plt.subplot(num_ejemplos, 3, i * 3 + 1)
        plt.imshow(imagenes[i].T, cmap="gray")
        plt.title("Imagen original")
        plt.axis("off")

        
        plt.subplot(num_ejemplos, 3, i * 3 + 2)
        plt.imshow(mascaras_reales[i].T, cmap="gray", interpolation='Nearest')
        plt.title("Máscara real")
        plt.axis("off")

        
        plt.subplot(num_ejemplos, 3, i * 3 + 3)
        plt.imshow(mascaras_predichas[i, ..., 0].T, cmap="gray", interpolation='Nearest')
        plt.title("Máscara predicha")
        plt.axis("off")

    plt.savefig("Resultado_UNet_Att.svg", format="svg")
    plt.tight_layout()
    plt.show()


mostrar_resultados(images_test, masks_test, predicciones, num_ejemplos=5)

GUARDAR MODELO

In [ ]:
# Guardar el modelo
model.save("/kaggle/working/Attention_Unet.keras")

from tensorflow.keras.models import load_model
modelo_cargado = load_model("/kaggle/working/Attention_Unet.keras")


modelo_cargado.summary()